In [1]:
from openmm import app, unit
import numpy as np
import openmm
import fresnel
import matplotlib.pyplot as plt
from simtk import unit

import sys
from polychrom.hdf5_format import HDF5Reporter
from polychrom import forcekits, forces, simulation

from matplotlib.cm import get_cmap
from matplotlib.colors import Normalize, to_rgb
import matplotlib.patches as mpatches

import polykit.polykit.renderers.backends as backends
from polykit.polykit.generators.initial_conformations import grow_cubic, create_random_walk
import polykit as polykit

from polychrom.lib.extrusion import bondUpdater

from polychrom.forces import heteropolymer_SSW

##run_homopolymer_sim.py input arguments
forcelist is a list of tuples. The first index in each tuple is the force you would like to add to the simulation (any force field in /src/polychrom/polychrom/forces.py or /src/polychrom/polychrom/legacy/forces.py), and the second index is keyword arguments for the force. 

##Default forces for worm-like chain homopolymer simulation
(forces.harmonic_bonds, {'bondLength': 1, 'bondWiggleDistance': 0.05})
(forces.angle_force, {'k':1.5})
(forces.spherical_confinement {'k':3})
(forces.polynomial_repulsive, {'trunc':3.0})


In [37]:
#describe what forces I want to test.

base_path = '/Users/kadendimarco/Desktop/Shrivinas_lab/genome_archetecture/chromosomes_openmm/polychrom_base/benchmarking/trajectory/homopolymer_test'

forcelist = [
    "[(forces.harmonic_bonds, {'bondLength': 1, 'bondWiggleDistance': 0.05}), (forces.angle_force, {'k':0.015})]",
]


trajectory_folder_list = [
    f'{base_path}/bonds_angle_angK0.015_r1',
]

# forcelist = [
#     "[(forces.harmonic_bonds, {'bondLength': 1, 'bondWiggleDistance': 0.05}), (forces.angle_force, {'k':0.15})]",
#     "[(forces.harmonic_bonds, {'bondLength': 1, 'bondWiggleDistance': 0.05}), (forces.angle_force, {'k':1.5})]",
#     "[(forces.harmonic_bonds, {'bondLength': 1, 'bondWiggleDistance': 0.05}), (forces.angle_force, {'k':15})]",
#     "[(forces.harmonic_bonds, {'bondLength': 1, 'bondWiggleDistance': 0.05}), (forces.angle_force, {'k':150})]"
# ]

# trajectory_folder_list = [
#     f'{base_path}/bonds_angle_angK0.15_r1',
#     f'{base_path}/bonds_angle_angK1.5_r1',
#     f'{base_path}/bonds_angle_angK15_r1',
#     f'{base_path}/bonds_angle_angK150_r1',
# ]



In [38]:
with open('simulation_homopolymer_commands.txt', 'w') as f:
    for force, folder in zip(forcelist, trajectory_folder_list):
        force_escaped = force.replace("'", "'\\''")
        cmd = f"python run_homopolymer_sim.py -o {folder} -f '{force_escaped}'\n"
        f.write(cmd)

print("Created simulation_homopolymer_commands.txt")

Created simulation_homopolymer_commands.txt


Now run cat simulation_homopolymer_commands.txt | parallel -j 4

#below code fits contact frequencies to polymer scaling laws

In [40]:
#tile output figures
import os
import glob
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

base_dir = "/Users/kadendimarco/Desktop/Shrivinas_lab/genome_archetecture/chromosomes_openmm/polychrom_base/benchmarking/trajectory/homopolymer_test"

# look for prefix corresponding to experiment
#if you want first 100 |i-j|
png_files = glob.glob(os.path.join(base_dir, "bonds_angle_angK*", "*100.png"))
#if you want all
#png_files = glob.glob(os.path.join(base_dir, "bonds_angle_angK*", "*w.png"))
png_files.sort() 

n_files = len(png_files)
n_cols = int(np.ceil(np.sqrt(n_files)))
n_rows = int(np.ceil(n_files / n_cols))

fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 4, n_rows * 4))
axes = axes.flatten()

for idx, png_file in enumerate(png_files):
    img = Image.open(png_file)
    axes[idx].imshow(img)
    axes[idx].axis('off')
    # Add title from directory name
    dir_name = os.path.basename(os.path.dirname(png_file))
    axes[idx].set_title(dir_name)

# Hide any unused subplots
for idx in range(n_files, len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.savefig('bonds_angle_angK_100_figure.png', dpi=300, bbox_inches='tight')
plt.close() 